# **AI TECH INSTITUTE** · *Intermediate AI & Data Science*
### Support Vector Classifier (SVC) - Interactive Deep Dive
**Instructor:** Amir Charkhi | **Dataset:** Breast Cancer Wisconsin (UCI)

---

## 📚 What You'll Learn

- How SVC finds the optimal decision boundary
- The role of support vectors
- Kernel trick for non-linear problems
- Hyperparameter tuning (C, gamma, kernel)
- Real-world application: Cancer diagnosis

---

## 🎯 The Big Idea: Maximum Margin Classifier

Imagine you're separating red and blue marbles on a table:

```
Bad separator:                Good separator (SVM):
                              
🔴 🔴 | 🔵 🔵                 🔴 🔴    |    🔵 🔵
🔴 🔴 | 🔵 🔵                 🔴 🔴    |    🔵 🔵
      ↑                                ↑
  Too close!                    Maximum margin!
```

**SVC finds the line (or hyperplane) that maximizes the distance to the nearest points from both classes.**

These nearest points are called **support vectors** - they "support" the decision boundary!

---

## 1. Setup

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.decomposition import PCA

# Plotly for interactive visualizations
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

---
## 2. Load Real Medical Data

**Dataset:** Breast Cancer Wisconsin (Diagnostic)

**Context:** Features computed from digitized images of breast mass:
- Radius, texture, perimeter, area, smoothness, etc.
- **Goal:** Predict if tumor is benign or malignant
- **569 patients**, 30 features

**Real-world impact:** Early cancer detection saves lives! 🎗️

In [ ]:
# Load data
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='diagnosis')

# Create readable labels
y_labels = y.map({0: 'Malignant', 1: 'Benign'})

# Display info
info_df = pd.DataFrame({
    'Metric': ['Total Samples', 'Features', 'Benign Cases', 'Malignant Cases', 'Class Balance'],
    'Value': [
        len(X),
        X.shape[1],
        (y == 1).sum(),
        (y == 0).sum(),
        f"{(y == 1).sum() / len(y):.1%} Benign"
    ]
})

print("🏥 Breast Cancer Wisconsin Dataset")
print("="*50)
for _, row in info_df.iterrows():
    print(f"{row['Metric']:.<30} {row['Value']}")
print("="*50)

In [ ]:
# Preview features
X.head()

### 📊 Explore Class Distribution

In [ ]:
# Interactive class distribution
class_counts = y_labels.value_counts()

fig = go.Figure(data=[
    go.Bar(
        x=class_counts.index,
        y=class_counts.values,
        text=class_counts.values,
        textposition='auto',
        marker_color=['#FF6B6B', '#4ECDC4']
    )
])

fig.update_layout(
    title='Class Distribution: Benign vs Malignant',
    xaxis_title='Diagnosis',
    yaxis_title='Number of Cases',
    template='plotly_white',
    height=400
)

fig.show()

---
## 3. Feature Relationships

Let's visualize how different tumor characteristics separate the classes.

In [ ]:
# Create visualization dataframe
viz_df = X[['mean radius', 'mean texture', 'mean area', 'mean smoothness']].copy()
viz_df['diagnosis'] = y_labels

# Interactive scatter matrix
fig = px.scatter_matrix(
    viz_df,
    dimensions=['mean radius', 'mean texture', 'mean area', 'mean smoothness'],
    color='diagnosis',
    title='Feature Relationships: Can We Separate the Classes?',
    height=700,
    color_discrete_map={'Benign': '#4ECDC4', 'Malignant': '#FF6B6B'}
)

fig.update_traces(diagonal_visible=False, showupperhalf=False)
fig.show()

**💡 Observation:** Some feature pairs show clear separation! SVC will find optimal boundaries.

---
## 4. Prepare Data for SVC

### 🎯 Critical Step: Feature Scaling

**Why scale for SVC?**
- SVC uses distances between points
- Features with large ranges dominate the calculation
- Example: `mean area` (range: 143-2501) vs `mean smoothness` (range: 0.05-0.16)

**Without scaling:** Area would dominate, smoothness would be ignored  
**With scaling:** All features equally important

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 📊 Visualize Scaling Impact

In [ ]:
# Compare before/after scaling
feature_name = 'mean radius'
feature_idx = list(X.columns).index(feature_name)

fig = make_subplots(rows=1, cols=2, subplot_titles=['Before Scaling', 'After Scaling'])

# Before scaling
fig.add_trace(
    go.Histogram(x=X_train[feature_name], name='Original', marker_color='lightblue'),
    row=1, col=1
)

# After scaling
fig.add_trace(
    go.Histogram(x=X_train_scaled[:, feature_idx], name='Scaled', marker_color='lightgreen'),
    row=1, col=2
)

fig.update_layout(title_text=f'Feature Scaling: {feature_name}', showlegend=False, height=400)
fig.show()

---
## 5. Linear SVC: When Data is Linearly Separable

### 🔍 Linear Kernel

Finds a straight line (or hyperplane) to separate classes:

```
Decision function: f(x) = w·x + b

Where:
  w = weights (normal vector to hyperplane)
  b = bias (intercept)
  
Prediction: sign(f(x))
  > 0 → Class 1
  < 0 → Class 0
```

### 📊 Hyperparameter: C (Regularization)

**C controls the trade-off:**
- **Small C (e.g., 0.1):** Wide margin, tolerates misclassifications → **Soft margin**
- **Large C (e.g., 100):** Narrow margin, fewer misclassifications → **Hard margin**

```
Small C:                 Large C:
🔴 🔴   |   🔵 🔵         🔴 🔴|🔵 🔵
🔴  🔵  |   🔵 🔵         🔴 🔴|🔵 🔵
   ↑                         ↑
Allows error             Strict boundary
```

In [ ]:
# Train Linear SVC
svc_linear = SVC(kernel='linear', C=1.0, random_state=42)
svc_linear.fit(X_train_scaled, y_train)

# Predictions
y_pred_linear = svc_linear.predict(X_test_scaled)
accuracy_linear = accuracy_score(y_test, y_pred_linear)

In [ ]:
# Confusion matrix visualization
cm = confusion_matrix(y_test, y_pred_linear)

fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Predicted Malignant', 'Predicted Benign'],
    y=['Actual Malignant', 'Actual Benign'],
    text=cm,
    texttemplate='%{text}',
    textfont={"size": 20},
    colorscale='Blues',
    showscale=False
))

fig.update_layout(
    title=f'Linear SVC Confusion Matrix<br>Accuracy: {accuracy_linear:.2%}',
    height=400,
    xaxis_title='Predicted',
    yaxis_title='Actual'
)

fig.show()

### 🎯 Support Vectors: The Key Players

**Support vectors** are the critical points closest to the decision boundary. Only these points determine the boundary position!

In [ ]:
# Number of support vectors
n_support = svc_linear.n_support_
total_support = svc_linear.support_vectors_.shape[0]

print(f"📊 Support Vectors Analysis")
print(f"   Total training samples: {len(X_train_scaled)}")
print(f"   Support vectors used: {total_support} ({total_support/len(X_train_scaled):.1%})")
print(f"   Class 0 (Malignant): {n_support[0]}")
print(f"   Class 1 (Benign): {n_support[1]}")
print(f"\n💡 Only {total_support/len(X_train_scaled):.1%} of training data determines the boundary!")

---
## 6. RBF Kernel: For Non-Linear Boundaries

### 🎨 The Kernel Trick

**Problem:** What if classes aren't linearly separable?

```
Linear won't work:        RBF kernel can handle:
                         
  🔴 🔵 🔴                   🔴  🔴  🔴
  🔵 🔴 🔵      →           🔵  🔵  🔵
  🔴 🔵 🔴                   🔴  🔴  🔴
```

**RBF (Radial Basis Function) kernel:**
- Creates circular/spherical decision boundaries
- Implicitly maps to higher dimensions
- Can capture complex patterns

### 📊 New Hyperparameter: Gamma

**Gamma controls the influence of each training point:**
- **Small gamma:** Far-reaching influence → **Smooth boundary**
- **Large gamma:** Local influence → **Complex boundary**

```
Small gamma (0.001):     Large gamma (10):
Smooth, general          Wiggly, specific
```

In [ ]:
# Train RBF SVC
svc_rbf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svc_rbf.fit(X_train_scaled, y_train)

# Predictions
y_pred_rbf = svc_rbf.predict(X_test_scaled)
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)

In [ ]:
# Compare Linear vs RBF
comparison = pd.DataFrame({
    'Kernel': ['Linear', 'RBF'],
    'Accuracy': [accuracy_linear, accuracy_rbf],
    'Support Vectors': [
        svc_linear.support_vectors_.shape[0],
        svc_rbf.support_vectors_.shape[0]
    ]
})

fig = go.Figure()

fig.add_trace(go.Bar(
    name='Accuracy',
    x=comparison['Kernel'],
    y=comparison['Accuracy'],
    text=[f"{x:.2%}" for x in comparison['Accuracy']],
    textposition='auto',
    marker_color=['#FF6B6B', '#4ECDC4']
))

fig.update_layout(
    title='Linear vs RBF Kernel Performance',
    yaxis_title='Accuracy',
    yaxis_range=[0.9, 1.0],
    template='plotly_white',
    height=400
)

fig.show()

---
## 7. Hyperparameter Tuning with Grid Search

Let's systematically find the best combination of C and gamma!

In [ ]:
# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1]
}

# Grid search with cross-validation
grid_search = GridSearchCV(
    SVC(kernel='rbf', random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

grid_search.fit(X_train_scaled, y_train)

In [ ]:
# Visualize grid search results
results = pd.DataFrame(grid_search.cv_results_)
pivot_table = results.pivot_table(
    values='mean_test_score',
    index='param_gamma',
    columns='param_C'
)

fig = go.Figure(data=go.Heatmap(
    z=pivot_table.values,
    x=[f'C={c}' for c in pivot_table.columns],
    y=[f'γ={g}' for g in pivot_table.index],
    text=np.round(pivot_table.values, 4),
    texttemplate='%{text:.4f}',
    colorscale='RdYlGn',
    colorbar=dict(title='Accuracy')
))

fig.update_layout(
    title='Grid Search Results: C vs Gamma',
    xaxis_title='C (Regularization)',
    yaxis_title='Gamma (Kernel Coefficient)',
    height=500
)

fig.show()

print(f"\n🏆 Best Parameters: C={grid_search.best_params_['C']}, gamma={grid_search.best_params_['gamma']}")
print(f"🎯 Best CV Accuracy: {grid_search.best_score_:.4f}")

### 🎯 Final Model with Best Parameters

In [ ]:
# Train with best parameters
best_svc = grid_search.best_estimator_
y_pred_best = best_svc.predict(X_test_scaled)
accuracy_best = accuracy_score(y_test, y_pred_best)

# Classification report
report = classification_report(y_test, y_pred_best, target_names=['Malignant', 'Benign'], output_dict=True)
report_df = pd.DataFrame(report).transpose()

print("📊 Final Model Performance")
print("="*60)
print(report_df.round(3))

---
## 8. Decision Boundary Visualization (2D Projection)

Since we have 30 features, let's use PCA to project to 2D and visualize the decision boundary.

In [ ]:
# Project to 2D using PCA
pca = PCA(n_components=2, random_state=42)
X_train_2d = pca.fit_transform(X_train_scaled)
X_test_2d = pca.transform(X_test_scaled)

# Train SVC on 2D data
svc_2d = SVC(kernel='rbf', C=grid_search.best_params_['C'], 
             gamma=grid_search.best_params_['gamma'], random_state=42)
svc_2d.fit(X_train_2d, y_train)

# Create mesh for decision boundary
x_min, x_max = X_train_2d[:, 0].min() - 1, X_train_2d[:, 0].max() + 1
y_min, y_max = X_train_2d[:, 1].min() - 1, X_train_2d[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))

# Predict on mesh
Z = svc_2d.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

In [ ]:
# Interactive decision boundary plot
fig = go.Figure()

# Decision boundary
fig.add_trace(go.Contour(
    x=xx[0],
    y=yy[:, 0],
    z=Z,
    colorscale=[[0, 'rgba(255,107,107,0.3)'], [1, 'rgba(78,205,196,0.3)']],
    showscale=False,
    hoverinfo='skip'
))

# Training points
for class_val, class_name, color in [(0, 'Malignant', '#FF6B6B'), (1, 'Benign', '#4ECDC4')]:
    mask = y_train == class_val
    fig.add_trace(go.Scatter(
        x=X_train_2d[mask, 0],
        y=X_train_2d[mask, 1],
        mode='markers',
        name=class_name,
        marker=dict(color=color, size=8, line=dict(width=1, color='white'))
    ))

# Support vectors
support_vectors_2d = X_train_2d[svc_2d.support_]
fig.add_trace(go.Scatter(
    x=support_vectors_2d[:, 0],
    y=support_vectors_2d[:, 1],
    mode='markers',
    name='Support Vectors',
    marker=dict(size=12, color='yellow', symbol='circle-open', line=dict(width=2, color='black'))
))

fig.update_layout(
    title=f'SVC Decision Boundary (2D PCA Projection)<br>Support Vectors: {len(svc_2d.support_)}',
    xaxis_title='First Principal Component',
    yaxis_title='Second Principal Component',
    template='plotly_white',
    height=600,
    hovermode='closest'
)

fig.show()

**💡 Key Observations:**
- Yellow circles = support vectors (the critical points)
- Shaded regions = predicted classes
- Boundary adapts to data structure
- Non-linear boundary captures complexity

---
## 9. Key Takeaways

### ✅ What We Learned:

**1. SVC Core Concepts:**
- Finds maximum margin between classes
- Only support vectors matter for the boundary
- Effective in high-dimensional spaces

**2. Kernels:**
- **Linear:** For linearly separable data, fast, interpretable
- **RBF:** For complex patterns, flexible, most popular
- **Polynomial:** For polynomial relationships
- **Sigmoid:** For neural network-like boundaries

**3. Hyperparameters:**
- **C:** Trade-off between margin width and misclassifications
  - Small C → Soft margin (more generalization)
  - Large C → Hard margin (less tolerance for errors)
- **Gamma (RBF):** Influence of each training point
  - Small gamma → Far-reaching influence
  - Large gamma → Local influence

**4. When to Use SVC:**
- ✅ High-dimensional data
- ✅ Clear margin between classes
- ✅ Small to medium datasets (< 100K samples)
- ✅ Need robust to outliers
- ❌ Very large datasets (slow to train)
- ❌ Need probability estimates (use `probability=True`, but slower)
- ❌ Real-time predictions with tight latency requirements

**5. Best Practices:**
- Always scale features!
- Use Grid Search for hyperparameter tuning
- Start with RBF kernel
- Check number of support vectors (if too many → overfitting)

---

### 🎯 Real-World Application:

Our SVC model achieved **{:.1%} accuracy** on breast cancer diagnosis:
- High precision for malignant cases (critical!)
- Robust decision boundary
- Only {:.0%} of training data (support vectors) needed

This demonstrates SVC's power for medical diagnosis where:
- Accuracy is critical
- Data is high-dimensional
- Clear separation exists

---

**AI Tech Institute** | *Building Tomorrow's AI Engineers Today*